In [1]:
import random

def binary_symmetric_channel(p, y):
    yy = vector(GF(2), y.length())
    nb_errs = 0
    for i in range(y.length()):
        r = random.random()
        if r < 1-p:
            yy[i] = 1-y[i]
            nb_errs += 1
        else:
            yy[i] = y[i]
    return yy, nb_errs

In [2]:
#construction de Gallager
def BinaryGallagerLDPC(n,k,wl,wc):
    K=GF(2)
    H1=[0 for i in range(n/wl)]
    v=[0]*n

    for i in range(wl):
        v[i] = 1

    for i in range(n/wl):
        H1[i] = vector(K, v[n-i*wl:]+v[:n-i*wl])

    H1=matrix(H1)
    H=[H1]
    H1t = H1.transpose()
    perm = [i for i in range(n)]
    for l in range(wc-1):
        Htmp = matrix(K,n,n/wl)
        random.shuffle(perm)
        for i in range(n):
            Htmp[i]=vector(K, H1t[perm[i]])
        H.append(Htmp.transpose())


    H = block_matrix(wc,1,H,subdivide=False)
    return H

H = BinaryGallagerLDPC(24, 12, 6, 3)

In [3]:
def vector_nonzero_ith_component(length, i, e, f):
    x = zero_vector(f, length)
    x[i] = e
    return x

def bit_flipping(H, y, max_iter):
    n = y.length()
    syndrome_inf = (H * y).hamming_weight()
    syndrome_weight = syndrome_inf
    canonic_base_vec = lambda i:  vector_nonzero_ith_component(n, i, 1, y.base_ring())

    for t in range(max_iter - 1):
        flip =  vector(GF(2), n)
        for i in range(n):
            s = (H * y)
            w = (H * (y + canonic_base_vec(i))).hamming_weight()
            if  w <= syndrome_inf and w < s.hamming_weight():
                if w < syndrome_inf:
                    for j in range(n):
                        flip[j] = 0
                    syndrome_inf = w
                flip[i] = 1
        y += flip
        
        
        if (H * y) == 0:
            return y

    return None

In [4]:
# Les noeuds d'information voisins d'un noeud de parité
def make_W(H):
    W = []
    for i in range(H.nrows()):
        tmp = []
        for j in range(H.ncols()):
            if H[i, j] == 1:
                tmp.append(j)
        W.append(tmp)
    return W

# Les noeuds de parité voisins d'un noeud d'information
def make_V(H):
    V = []
    for i in range(H.ncols()):
        tmp = []
        for j in range(H.nrows()):
            if H[j, i] == 1:
                tmp.append(j)
        V.append(tmp)
    return V

# traduction littérale de l'algo donné en cours
def belief_propagation(H, y, p_err_canal, max_iter):
    n = H.ncols()
    k = n - H.nrows()
    
    q = Matrix(RR, n, n - k, sparse = True)
    r = Matrix(RR, n - k, n, sparse = True)
    
    # Initialisation des p_i = Proba [ c_i = 1 | y_i ]
    p = vector(RR, n)
    for i in range(n):
        if y[i] == 0:
            p[i] = p_err_canal
        else:
            p[i] = 1. - p_err_canal
        # On initialise q_{i,j} avec p_i
        for j in range(n - k):
            q[i, j] = p[i]
    
    c = vector(GF(2), n)
    W = make_W(H)
    V = make_V(H)
    for _ in range(max_iter - 1):
        # Etape horizontale
        for j in range(n - k):
            for i in W[j]:
                r[j, i] = (1. - prod([0 if s == i else 1. - 2. * q[s, j] for s in W[j]])) / 2.
              
        # Etape verticale
        for i in range(n):
            for j in V[i]:
                p1 = p[i]
                p2 = 1. - p[i]
                for s in V[i]:
                    if s != j:
                        p1 *= r[s, i]
                        p2 *= (1. - r[s, i])
                q[i, j] = p1 / (p1 + p2)
                
        # décision
        for i in range(n):
            p1 = p[i]
            p2 = 1. - p[i]
            
            for j in V[i]:
                p1 *= r[j, i]
                p2 *= (1. - r[j, i])
            q_i = p1 / (p1 + p2)
            c[i] = 1 if q_i > 1 / 2 else 0
        
        # c appartient au code ?
        if (H * c).hamming_weight() == 0:
            return c
        
    return None


In [5]:
def belief_propagation_llr(H, y, p_err_canal, iter_max):
    n = y.length()
    k = n - H.nrows()
   
    V = make_V(H)
    W = make_W(H)
    q = Matrix(RR, n, n - k, sparse = True)
    r = Matrix(RR, n - k, n, sparse = True)
    
    # Initialisation des LLR p_i = log(Proba [ c_i = 1 | y_i ] / Proba [ c_i = 0 | y_i ])
    p = vector(RR, n)
    for i in range(n):
        if y[i] == 0:
            p[i] = p_err_canal
        else:
            p[i] = 1. - p_err_canal
        p[i] = log(p[i] / (1 - p[i]), 2)
        for j in range(n - k):
            q[i, j] = p[i]
            
    c = vector(GF(2), n)
    W = make_W(H)
    V = make_V(H)
    for _ in range(iter_max - 1):
        # Etape horizontale
        for j in range(n - k):
            for i in W[j]:
                sg = 1
                s = 0.
                for s in W[j]:
                    if s != i:
                        sg *= sign(q[s, j])
                        s -= log(tanh(abs(q[s, j]) / 2.), 2)
                r[j,i] = -sg * log(tanh(s / 2.), 2)
        
        # Etape verticale
        for i in range(n):
            for j in V[i]:
                s = p[i]
                q[i, j] = p[i] + sum([0 if s == j else r[s, j] for s in V[i]])
                
        # Décision
        for i in range(n):
            q_i = p[i] + sum([r[j, i] for j in V[i]])
            c[i] = 0 if q_i < 0 else 1
        
        # c appartient au code ?
        if (H * c).hamming_weight() == 0:
            return c
    
    return None

In [ ]:
import time


C = H.right_kernel()
y = C.random_element()

print("y = ")
print(y)
p_succ = .6
noisy, errs = binary_symmetric_channel(p_succ, y)

print("noisy = ", noisy)
print("nb_errs=", errs)

t = time.time()
dec = bit_flipping(H, noisy,100)
tBF = time.time() - t
print("decode bit flipping:", tBF, "s.")

iter_max = 100
t = time.time()
dec = belief_propagation(H, y, 1-p_succ, 100)
print("Success: ", dec == y)

tBP = time.time() - t
print("decode belief propagation:", tBP, "s.")

print("Success: ", dec == y)

t = time.time()
dec = belief_propagation_llr(H, noisy, 1-p_succ, 100)

tBP_LLR = time.time() - t

print("decode belief propagation (variante LLR):", tBP_LLR, "s.")

print("Success: ", dec == y)

print("BP speedup over bit flipping: {:.2f}x".format(tBF/tBP)) 
print("BP with LLR speedup over BP: {:.2f}x".format(tBP/tBP_LLR))

bf_times = vector(RR, 100)
bp_times = vector(RR, 100)
bp_llr_times = vector(RR, 100)
p_succ = .6
for i in range(100):
    
    y = C.random_element()
    noisy, errs = binary_symmetric_channel(p_succ, y)

    t = time.time()
    dec = bit_flipping(H, noisy,100)
    tBF = time.time() - t

    iter_max = 100
    t = time.time()
    dec = belief_propagation(H, y, 1-p_succ, 100)

    tBP = time.time() - t

    t = time.time()
    dec = belief_propagation_llr(H, noisy, 1-p_succ, 100)

    tBP_LLR = time.time() - t

    bf_times[i] = tBF
    bp_times[i] = tBP
    bp_llr_times[i] = tBP_LLR

import sage.stats.basic_stats
print("BF: mean exec time:", mean(bf_times), "s , median exec time:", median(bf_times))

print("BP: mean exec time:", mean(bp_times), "s , median exec time:", median(bp_times))

print("BP LLR: mean exec time:", mean(bp_llr_times), "s , median exec time:", median(bp_llr_times))

y = 
(1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1)
noisy =  (1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0)
nb_errs= 11
decode bit flipping: 0.05742621421813965 s.
Success:  True
decode belief propagation: 0.00452876091003418 s.
Success:  True
decode belief propagation (variante LLR): 0.8861441612243652 s.
Success:  False
BP speedup over bit flipping: 12.68x
BP with LLR speedup over BP: 0.01x
